Download libraries

In [16]:
import sys
import os
import time
import csv

In [70]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/imacair/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.cross_validation import train_test_split
import numpy as np

/Users/imacair/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

Set working directory

In [6]:
print os.getcwd();

/Users/imacair


Reads the file

In [7]:
os.chdir("/Users/imacair/Desktop/Products3/")

In [8]:
data = pd.read_csv('Final_Manual_0805.csv', encoding= "latin-1",delimiter=',',low_memory=False)

In [9]:
#Converts text into ASCII
data.message = data.message.str.encode('ascii','replace')
data.sentiment = data.sentiment.str.encode('utf-8','replace')


In [10]:
len(data)

965

Take a sample from data

In [11]:
data_t=data["message"]

In [12]:
#lowercase before abbriviation translation
data_t = data_t.str.lower()

In [13]:
#takes sentiment
data_s=data["sentiment"]

Preprocess the data

Replacement of abbriviations

In [17]:
with open('abbrev.csv', mode='r') as infile:
    reader = csv.reader(infile)
    replacement = {rows[0].lower():rows[1].lower() for rows in reader              
                  }

In [18]:
##How in works
s1 = 'y r u l8'

s2 = ' '.join([replacement.get(w, w) for w in s1.split()])
s2

'why? -or- yes are you l8'

In [19]:
result = pd.DataFrame()
result = data_t

In [20]:
for i in range(len(result)):
    data_t.values[i]=' '.join([replacement.get(w, w) for w in data_t.values[i].split()])

In [22]:
data_t.head()

0    #llbite ankylosing spondylitis - have (in date...
1    #as 3 things people with ankylosing spondyliti...
2    working group on the assessment of toxic chemi...
3    i dont know if intense text sex the medication...
4    #imaginedragons singer @danreynolds opens up a...
Name: message, dtype: object

In [23]:
type(result)

pandas.core.series.Series

In [24]:
data_t=result

In [25]:
#lowercase
data_t = data_t.str.lower()
#Remove urls
data_t= data_t.str.replace(r'(http.*) |(http.*)$|\n', "",)
#Remove twitter handles
data_t = data_t.str.replace(r"@\\w+", "")
#remove htmls
data_t = data_t.str.replace(r'<.*?>', "")
#Remove citations
data_t = data_t.str.replace(r'@[a-zA-Z0-9]*', "")

#remove _
#data_t = data_t.str.replace(r'\_+',"")

In [28]:
#Start of sentiment analysis

In [71]:
analyzer = SentimentIntensityAnalyzer()

In [80]:
analyzer.polarity_scores(data_t[190])

{'compound': -0.0772, 'neg': 0.071, 'neu': 0.929, 'pos': 0.0}

In [106]:
res= pd.DataFrame( index=range(0,len(data_t)),columns = {'SentimentVader'} )

In [107]:
for i in range(len(data_t)):
    vs = analyzer.polarity_scores(data_t.values[i])
    if ((vs['compound']==0)):
        res.values[i]= 'neu' 
    elif ((vs['compound'] < 0)):
        res.values[i]= 'neg'
    else:
        res.values[i]= 'pos'
    

In [108]:
res.SentimentVader

0      neg
1      neu
2      neu
3      neg
4      neg
5      neu
6      neg
7      neg
8      pos
9      neu
10     neu
11     neu
12     neu
13     neg
14     neu
15     neu
16     neg
17     neg
18     neu
19     neg
20     pos
21     neu
22     neg
23     neg
24     neu
25     pos
26     neu
27     pos
28     neu
29     pos
      ... 
935    pos
936    pos
937    pos
938    pos
939    pos
940    neu
941    neu
942    pos
943    pos
944    neg
945    pos
946    pos
947    neu
948    pos
949    pos
950    pos
951    pos
952    neg
953    neg
954    pos
955    pos
956    pos
957    pos
958    pos
959    neg
960    neg
961    neu
962    pos
963    pos
964    neu
Name: SentimentVader, dtype: object

In [40]:
data_t.head(10)

0    #llbite ankylosing spondylitis - have (in date...
1    #as 3 things people with ankylosing spondyliti...
2    working group on the assessment of toxic chemi...
3    i dont know if intense text sex the medication...
4    #imaginedragons singer  opens up about his str...
5    chinese medicine cure ankylosing spondylitis c...
6    pain relief for patients with rheumatoid arthr...
7    suffering from #ankylosingspondylitis? check o...
8    herself goodwill newsletter significant other ...
9    how they did it: 5 people with ankylosing spon...
Name: message, dtype: object

In [109]:
from textblob import TextBlob

In [110]:
final=pd.concat([data_t, res], axis=1)
final=pd.concat([final, data_s], axis=1)

In [111]:
res5= pd.DataFrame( index=range(0,len(data_t)),columns = {'new_sent'} )
res5[(final.sentiment==u'2')] = 'neg'
res5[(final.sentiment==u'1')] = 'neg'
res5[(final['sentiment']==u'3')] = 'pos'
res5[(final['sentiment']==u'4')] = 'pos'
res5[(final['sentiment']==u'N')] = 'neu'
final=pd.concat([final, res5], axis=2)

In [112]:
final

,message,SentimentVader,sentiment,new_sent
0,#llbite ankylosing spondylitis - have (in date...,neg,2,neg
1,#as 3 things people with ankylosing spondyliti...,neu,3,pos
2,working group on the assessment of toxic chemi...,neu,3,pos
3,i dont know if intense text sex the medication...,neg,1,neg
4,#imaginedragons singer opens up about his str...,neg,2,neg
5,chinese medicine cure ankylosing spondylitis c...,neu,3,pos
6,pain relief for patients with rheumatoid arthr...,neg,3,pos
7,suffering from #ankylosingspondylitis? check o...,neg,3,pos
8,herself goodwill newsletter significant other ...,pos,2,neg
9,how they did it: 5 people with ankylosing spon...,neu,3,pos


In [113]:
from sklearn.metrics import confusion_matrix

In [114]:
print(confusion_matrix(final.new_sent, final.SentimentVader).transpose())

[[147  44  43]
 [104 126 103]
 [ 74  98 226]]
